In [6]:
import marvin

from random import randint

import matplotlib.pyplot as plt
import marvin.utils.plot.map as mapplot
import numpy as np
import pandas as pd

from newer_Galaxy import SpiralGalaxy
from copy import deepcopy
from marvin.tools import Maps

from matplotlib.lines import Line2D
from matplotlib.patches import Patch

In [7]:
plt.style.use('ggplot')
plt.style.use('seaborn-colorblind')

In [8]:
import os

def append_directory(path):
    fits_list = []
    
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith(".fits.gz"):
                element = str(path) + str(file)
                fits_list.append(element)
    return fits_list

In [9]:
def append_usable_galaxies(fits_list):
    usable_galaxy_list = []
    
    for path in fits_list:
        try:
            try:
                gal = SpiralGalaxy(path)
            except:
                gal = SpiralGalaxy(path)
        except:
            print("{} failed to load, it  may have not been processed in MPL-6. ¯\_(ツ)_/¯ Check the SDSS track website to make sure.".format(gal.mangaid))
            continue
            
        if gal.check_usability():
            usable_galaxy_list.append(path)
            
    return usable_galaxy_list

In [10]:
gal_list = append_directory('/home/sshamsi/sas/mangawork/manga/sandbox/galaxyzoo3d/v1_0_0/')

In [12]:
#usable_gals = append_usable_galaxies(gal_list)

usable_gals = np.load('usable_path_list.npy')

In [9]:
def make_random_plots(num_gals, test_gal_list):
    run_frac = num_gals/6
    num_runs = int(run_frac)
    last_run = 6 * (run_frac - num_runs)
    
    for j in range(num_runs):
        fig, axes = plt.subplots(6, 4, figsize=(32, 6 * 8))
        
        for i in range(6):
            gal = BarGalaxy(test_gal_list.pop(random.randint(0, len(test_gal_list)-1)))
            
            img_ax = axes[i, 0]
            img_ax.axis('off')
            img_ax.add_patch(gal.data.get_hexagon(correct_hex=True, edgecolor='C4'))
            img_ax.imshow(gal.data.image)
            
            ax_list = axes[i][1:]
            
            gal.make_hamap_bar_masks()
            
            cbrange = gal.hamap.plot(return_cbrange=True)
            
            mask_list = [gal.hamap.mask, gal.hamap_bar_mask, gal.hamap_non_bar_mask]
            
            for ax, mask in zip(ax_list, mask_list):
                mapplot.plot(dapmap = gal.hamap, fig=fig, ax=ax, mask = mask, cbrange=cbrange, title = str(gal.mangaid))
                
        fig.savefig(str(j) + '.png')
        
    fig, axes = plt.subplots(last_run, 4, figsize=(32, last_run * 8))
    
    for k in range(j, j + last_run):
        gal = BarGalaxy(test_gal_list.pop(random.randint(0, len(test_gal_list)-1)))

        img_ax = axes[i, 0]
        img_ax.axis('off')
        img_ax.add_patch(gal.data.get_hexagon(correct_hex=True, edgecolor='C4'))
        img_ax.imshow(gal.data.image)
        
        ax_list = axes[i][1:]
        
        gal.make_hamap_bar_masks()
        
        cbrange = gal.hamap.plot(return_cbrange=True)

        mask_list = [gal.hamap.mask, gal.hamap_bar_mask, gal.hamap_non_bar_mask]

        for ax, mask in zip(ax_list, mask_list):
            mapplot.plot(dapmap = gal.hamap, fig=fig, ax=ax, mask = mask, cbrange=cbrange, title = str(gal.mangaid))

In [ ]:
tot_sfr_vals = []
spiral_sfr_vals = []
non_spiral_sfr_vals = []

tot_sfr_stdv = []
spiral_sfr_stdv = []
non_spiral_sfr_stdv = []

sfr_dapall = []

for gal in gal_objs:
    tot_sfr_vals.append(gal.get_integrated_sfr()[0])
    tot_sfr_stdv.append(gal.get_integrated_sfr()[1])
    
    spiral_sfr_vals.append(gal.get_integrated_sfr(mode='spirals')[0])
    spiral_sfr_stdv.append(gal.get_integrated_sfr(mode='spirals')[1])
    
    non_spiral_sfr_vals.append(gal.get_integrated_sfr(mode='non-spirals')[0])
    non_spiral_sfr_stdv.append(gal.get_integrated_sfr(mode='non-spirals')[1])
    
    sfr_dapall.append(gal.maps.dapall['sfr_1re'])

In [ ]:
calc_dap_ratio = []

for i in range(len(gal_objs)):
    ratio = tot_sfr_vals[i]/sfr_dapall[i]
    if ratio < 25:
        calc_dap_ratio.append(ratio)

In [ ]:
fig = plt.figure(figsize = (16, 12))

plt.hlines(y=1, xmin=0, xmax=len(calc_dap_ratio), linestyle = '--', label = '$y=1$')
plt.scatter(range(len(calc_dap_ratio)), calc_dap_ratio, color='g', marker='x', label='Galaxy')

lobf = np.polyfit(range(len(calc_dap_ratio)), calc_dap_ratio, 1)
plt.plot(range(len(calc_dap_ratio)), np.poly1d(lobf)(range(len(calc_dap_ratio))), label = '$y = {} x + {}$'.format(lobf[0], lobf[1]), color='purple')

plt.xticks([], [])

plt.ylim(0, 25)

plt.ylabel('Ratio\n$SFR_{calculated}/SFR_{DAPall}$', rotation=0, labelpad=80)
plt.xlabel('Galaxies', labelpad = 20)

plt.legend()

In [ ]:
sfr_pc_spiral = []
sfr_pc_spiral_stdv = []

for i in range(len(gal_objs)):
    pc = 100 * spiral_sfr_vals[i]/tot_sfr_vals[i]
    sfr_pc_spiral.append(pc)
    
    sfr_pc_spiral_stdv.append(pc * ((spiral_sfr_stdv[i]/spiral_sfr_vals[i]) + (tot_sfr_stdv[i]/tot_sfr_vals[i])))

In [ ]:
fig = plt.figure(figsize=(16, 12))

plt.hist(sfr_pc_spiral, bins = 20)

plt.xlabel('Percent SFR in Spiral Arms', labelpad=20)
plt.ylabel('Num Galaxies', rotation=0, labelpad=60)
plt.title('20 Bins')

In [ ]:
mass_list = []
mangaid_list = []

for gal in gal_objs:    
    mass_list.append(gal.get_lgmass())
    mangaid_list.append(gal.mangaid)

In [ ]:
main_df = pd.DataFrame()

In [ ]:
main_df['mangaid'] = mangaid_list
main_df['tot_sfr'] = tot_sfr_vals
main_df['sig_tot_sfr'] = tot_sfr_stdv
main_df['spiral_sfr'] = spiral_sfr_vals
main_df['sig_spiral_sfr'] = spiral_sfr_stdv
main_df['nonspiral_sfr'] = non_spiral_sfr_vals
main_df['sig_nonspiral_sfr'] = non_spiral_sfr_stdv
main_df['dap_sfr'] = sfr_dapall
main_df['lgmass'] = mass_list

In [ ]:
main_df

In [ ]:
labels = np.arange(1, 11)

main_df['mass_bin'] = pd.qcut(main_df['lgmass'], len(labels), labels = labels)

In [ ]:
main_df

In [ ]:
main_df['pc_in_spirals'] = sfr_pc_spiral
main_df['sig_pc_in_spirals'] = sfr_pc_spiral_stdv

In [ ]:
avg_pc_spirals_binned = main_df.drop(82).groupby('mass_bin').pc_in_spirals.mean().values

In [ ]:
main_df['sig2_pc_in_spirals'] = main_df.sig_pc_in_spirals ** 2
sig_avg_pc_spirals_binned = (main_df.drop(82).groupby('mass_bin').sig2_pc_in_spirals.mean() ** 0.5).values

In [ ]:
main_df

In [ ]:
output, qcut_bins = pd.qcut(main_df.lgmass, len(labels), labels = labels, retbins=True)

In [ ]:
qcut_bins

In [ ]:
for i in range(len(qcut_bins)-1):
    print('"${0:.2f}$",'.format((qcut_bins[i] + qcut_bins[i + 1])/2))

In [ ]:
xtick_labels = ["$9.40$\n12 Galaxies", "$9.89$\n11 Galaxies", "$10.09$\n11 Galaxies", "$10.26$\n11 Galaxies",
                "$10.38$\n11 Galaxies", "$10.48$\n11 Galaxies", "$10.61$\n11 Galaxies", "$10.76$\n11 Galaxies",
                "$10.86$\n11 Galaxies", "$11.13$\n12 Galaxies",]

In [ ]:
xtick_labels1 = ["$9.40$\n12 Galaxies", "$10.09$\n11 Galaxies", "$10.48$\n11 Galaxies",
                 "$10.76$\n11 Galaxies", "$11.13$\n12 Galaxies",]

In [ ]:
np.arange(1, 11, 2)

In [ ]:
fig = plt.figure(figsize=(12 * 1.1, 9 * 1.1))

plt.plot(labels, avg_pc_spirals_binned, color='red')
plt.scatter(labels, avg_pc_spirals_binned, color='red')

plt.errorbar(labels, avg_pc_spirals_binned, fmt='none', elinewidth=2, ecolor='red', capsize=10, capthick=2,
             yerr=sig_avg_pc_spirals_binned)

plt.xlabel('Mass Bins $log_{10}(M_☉)$', labelpad=30, fontsize=50)
plt.xticks(np.arange(1, 11, 2), labels=xtick_labels1, fontsize=20)
plt.yticks(fontsize = 25)
plt.ylabel('% Total SFR in Spiral Arms', fontsize=50)

In [ ]:
gal_dfs = []

for gal in gal_objs:
    gal.spax_df['lgmass'] = gal.get_lgmass()
    gal_dfs.append(gal.spax_df)

In [ ]:
main_spax_df = pd.concat(gal_dfs, ignore_index=True)

In [ ]:
main_spax_df

In [ ]:
main_spax_df['rad_bin'] = pd.cut(main_spax_df.r_re, np.arange(0, 1.1, 0.1), labels = np.arange(1, 11))

In [ ]:
main_spax_df['lgmass_bin'] = pd.qcut(main_spax_df['lgmass'], 3, labels = [1, 2, 3])

In [ ]:
out, bins = pd.qcut(main_spax_df['lgmass'], 3, labels = [1, 2, 3], retbins=True)

In [ ]:
bins

In [ ]:
for i in range(len(bins)-1):
    print((bins[i]+bins[i+1])/2)

In [ ]:
main_spax_df = main_spax_df.dropna()

In [ ]:
main_spax_df = main_spax_df[2 * main_spax_df.sig_sfr < main_spax_df.sfr]

In [ ]:
main_spax_df['sig2_sfr'] = (main_spax_df.sig_sfr ** 2)

In [ ]:
main_spax_df

In [ ]:
main_spax_df.groupby('rad_bin').mean()

In [ ]:
radbin_spiral_sfr1 = main_spax_df[(main_spax_df.spaxel_type == 'Spiral') & (main_spax_df.lgmass_bin==1)].groupby('rad_bin').sfr.mean().values
sig_radbin_spiral_sfr1 = (main_spax_df[(main_spax_df.spaxel_type == 'Spiral') & (main_spax_df.lgmass_bin==1)].groupby('rad_bin').sig2_sfr.mean().values) ** 0.5

radbin_spiral_sfr2 = main_spax_df[(main_spax_df.spaxel_type == 'Spiral') & (main_spax_df.lgmass_bin==2)].groupby('rad_bin').sfr.mean().values
sig_radbin_spiral_sfr2 = (main_spax_df[(main_spax_df.spaxel_type == 'Spiral') & (main_spax_df.lgmass_bin==2)].groupby('rad_bin').sig2_sfr.mean().values) ** 0.5

radbin_spiral_sfr3 = main_spax_df[(main_spax_df.spaxel_type == 'Spiral') & (main_spax_df.lgmass_bin==3)].groupby('rad_bin').sfr.mean().values
sig_radbin_spiral_sfr3 = (main_spax_df[(main_spax_df.spaxel_type == 'Spiral') & (main_spax_df.lgmass_bin==3)].groupby('rad_bin').sig2_sfr.mean().values) ** 0.5


radbin_nspiral_sfr1 = main_spax_df[(main_spax_df.spaxel_type == 'Non Spiral') & (main_spax_df.lgmass_bin==1)].groupby('rad_bin').sfr.mean().values
sig_radbin_nspiral_sfr1 = (main_spax_df[(main_spax_df.spaxel_type == 'Non Spiral') & (main_spax_df.lgmass_bin==1)].groupby('rad_bin').sig2_sfr.mean().values) ** 0.5

radbin_nspiral_sfr2 = main_spax_df[(main_spax_df.spaxel_type == 'Non Spiral') & (main_spax_df.lgmass_bin==2)].groupby('rad_bin').sfr.mean().values
sig_radbin_nspiral_sfr2 = (main_spax_df[(main_spax_df.spaxel_type == 'Non Spiral') & (main_spax_df.lgmass_bin==2)].groupby('rad_bin').sig2_sfr.mean().values) ** 0.5

radbin_nspiral_sfr3 = main_spax_df[(main_spax_df.spaxel_type == 'Non Spiral') & (main_spax_df.lgmass_bin==3)].groupby('rad_bin').sfr.mean().values
sig_radbin_nspiral_sfr3 = (main_spax_df[(main_spax_df.spaxel_type == 'Non Spiral') & (main_spax_df.lgmass_bin==3)].groupby('rad_bin').sig2_sfr.mean().values) ** 0.5

In [ ]:
fig = plt.figure(figsize=(12, 9))

x = np.arange(10)

plt.errorbar(x, radbin_spiral_sfr1, fmt='-b', elinewidth=3, ecolor='b', capsize=0, yerr=sig_radbin_spiral_sfr1)
plt.errorbar(x, radbin_nspiral_sfr1, fmt='--b', elinewidth=1, ecolor='b', capsize=0, yerr=sig_radbin_nspiral_sfr1)

#plt.scatter(x, radbin_spiral_sfr1, marker='X', color='blue')
#plt.scatter(x, radbin_nspiral_sfr1, marker='X', color='blue')



plt.errorbar(x, radbin_spiral_sfr2, fmt='-g', elinewidth=3, ecolor='g', capsize=0, yerr=sig_radbin_spiral_sfr2)
plt.errorbar(x, radbin_nspiral_sfr2, fmt='--g', elinewidth=1, ecolor='g', capsize=0, yerr=sig_radbin_nspiral_sfr2)

#plt.scatter(x, radbin_spiral_sfr2, marker='o', color='g')
#plt.scatter(x, radbin_nspiral_sfr2, marker='o', color='g')


plt.errorbar(x, radbin_spiral_sfr3, fmt='-r', elinewidth=3, ecolor='r', capsize=0, yerr=sig_radbin_spiral_sfr3)
plt.errorbar(x, radbin_nspiral_sfr3, fmt='--r', elinewidth=1, ecolor='r', capsize=0, yerr=sig_radbin_nspiral_sfr3)

#plt.scatter(x, radbin_spiral_sfr3, marker='^', color='r')
#plt.scatter(x, radbin_nspiral_sfr3, marker='^', color='r')

'''
plt.plot(x, radbin_spiral_sfr2, '-g')
plt.plot(x, radbin_nspiral_sfr2, '--g')

plt.plot(x, radbin_spiral_sfr3, '-r')
plt.plot(x, radbin_nspiral_sfr3, '--r')
'''

plt.xlabel('$r/r_e$', fontsize=50, labelpad=15)
plt.ylabel('Average SFR\n$ M_☉year^{-1}Kpc^{-2}$', fontsize=50, labelpad=15)

plt.xticks([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], fontsize=30)
plt.yticks(fontsize=30)

custom_lines = [Line2D([0], [0], marker='s', color='blue', linestyle='none', markersize=10),
                Line2D([0], [0], marker='s', color='green', linestyle='none', markersize=10),
                Line2D([0], [0], marker='s', color='red', linestyle='none', markersize=10),
                Line2D([0], [0], linestyle='-', color = 'black', lw=2),
                Line2D([0], [0], linestyle='--', color = 'black', lw=2)]

#plt.legend(custom_lines, ['22153 Spaxels (9.018 - 10.156 $M_☉$)', '20849 Spaxels (10.156 - 10.590 $M_☉$)',
 #                         '19727 Spaxels (10.590 - 11.348 $M_☉$)', 'Inside Spiral Arms',
  #                        'Outside Spiral Arms'], prop={'size': 25})

In [ ]:
radbin_spiral_sfr = main_spax_df[main_spax_df.spaxel_type == 'Spiral'].groupby('rad_bin').sfr.mean().values
sig_radbin_spiral_sfr = (main_spax_df[main_spax_df.spaxel_type == 'Spiral'].groupby('rad_bin').sig2_sfr.mean().values) ** 0.5

radbin_nspiral_sfr = main_spax_df[main_spax_df.spaxel_type == 'Non Spiral'].groupby('rad_bin').sfr.mean().values
sig_radbin_nspiral_sfr = (main_spax_df[main_spax_df.spaxel_type == 'Non Spiral'].groupby('rad_bin').sig2_sfr.mean().values) ** 0.5

In [ ]:
len(main_spax_df[main_spax_df.lgmass_bin==1])

In [ ]:
len(main_spax_df[main_spax_df.lgmass_bin==2])

In [ ]:
len(main_spax_df[main_spax_df.lgmass_bin==3])

In [ ]:
gal_boi = SpiralGalaxy('/raid5/homes/sshamsi/sas/mangawork/manga/sandbox/galaxyzoo3d/v1_0_0/1-38550_127_5679053.fits.gz')

fig, axes = plt.subplots(4, 1, figsize=(8, 32))
img_ax = axes[0]
img_ax.axis('off')
img_ax.add_patch(gal_boi.data.get_hexagon(correct_hex=True, edgecolor='C4'))
img_ax.imshow(gal_boi.data.image)

ax_list = axes[1:]

gal_boi.make_hamap_spiral_masks()
            
cbrange = gal_boi.hamap.plot(return_cbrange=True)
            
mask_list = [gal_boi.hamap.mask, gal_boi.hamap_spiral_mask, gal_boi.hamap_non_spiral_mask]
            
for ax, mask in zip(ax_list, mask_list):
    mapplot.plot(dapmap = gal.hamap, fig=fig, ax=ax, mask = mask, cbrange=cbrange, title = 'H-α Flux Map')

In [ ]:
for gal in gal_objs:
    boolean = np.zeros(gal.hamap.shape, dtype=bool)
    
    a, b = boolean.shape
    j, i = (a-1)/2, (b-1)/2
    
    for y, x in [(y, x) for y in range(a) for x in range(b)]:
        if ((y - j)**2 + (x - i)**2) ** 0.5 < 4:
            boolean[y, x] = True
    
    centre_mask = boolean * 2**30
    gal.hamap.plot(mask=centre_mask)

In [ ]:
def central_hole(test_gal_list, size):
    run_frac = len(test_gal_list)/6
    num_runs = int(run_frac)
    last_run = 6 * (run_frac - num_runs)
    
    for j in range(num_runs):
        fig, axes = plt.subplots(6, 2, figsize=(8 * 2, 6 * 8))
        
        for i in range(6):
            gal = SpiralGalaxy(test_gal_list.pop(random.randint(0, len(test_gal_list)-1)))
            cbrange = gal.hamap.plot(return_cbrange=True)
            gal.make_hamap_spiral_masks()
            
            mapplot.plot(dapmap = gal.hamap, fig=fig, ax=axes[i, 0], cbrange=cbrange, title = str(gal.mangaid))
            
            boolean = np.zeros(gal.hamap.shape, dtype=bool)
            
            a, b = boolean.shape
            j, i = (a-1)/2, (b-1)/2
            
            for y, x in [(y, x) for y in range(a) for x in range(b)]:
                if ((y - j)**2 + (x - i)**2) ** 0.5 < size:
                    boolean[y, x] = True
                    
            centre_hole = boolean * 2**30
            centre_mask = deepcopy(gal.hamap.mask) | centre_hole
            
            mapplot.plot(dapmap = gal.hamap, fig=fig, ax=axes[i, 1], mask = centre_mask, cbrange=cbrange, title = str(gal.mangaid))
                                    
                
        fig.savefig(str(j) + '.png')
        
    fig, axes = plt.subplots(last_run, 2, figsize=(8 * 2, 8 * last_run))
    
    for k in range(num_runs, num_runs + last_run):
        gal = SpiralGalaxy(test_gal_list.pop(random.randint(0, len(test_gal_list)-1)))
        cbrange = gal.hamap.plot(return_cbrange=True)
        gal.make_hamap_spiral_masks()
        
        mapplot.plot(dapmap = gal.hamap, fig=fig, ax=axes[k - num_runs, 0], cbrange=cbrange, title = str(gal.mangaid))
            
        boolean = np.zeros(gal.hamap.shape, dtype=bool)
            
        a, b = boolean.shape
        j, i = (a-1)/2, (b-1)/2

        for y, x in [(y, x) for y in range(a) for x in range(b)]:
            if ((y - j)**2 + (x - i)**2) ** 0.5 < size:
                boolean[y, x] = True

        centre_hole = boolean * 2**30
        centre_mask = deepcopy(gal.hamap.mask) | centre_hole

        mapplot.plot(dapmap = gal.hamap, fig=fig, ax=axes[k - num_runs, 1], mask = centre_mask, cbrange=cbrange, title = str(gal.mangaid))                    
                
        fig.savefig(str(k) + '.png')

In [ ]:
central